<a href="https://colab.research.google.com/github/sourcesync/kagglex_gemma/blob/gw%2Finitial/colab/few_shot_gemma_huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This noteboook demonstrates the following:
* ...

# Install required packages

In [15]:
!pip install langchain langchain-community langchainhub langchain_huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 15.3 MB/s eta 0:00:00


# Import required packages

In [17]:
import os

# For logging into Hugging Face model hub
from huggingface_hub import login

# For accessing user secret keys on Kaggle
from google.colab import userdata
#from kaggle_secrets import UserSecretsClient

# Import module for accessing Hugging Face language models.
# from langchain_community.llms import HuggingFaceEndpoint # Deprecated
from langchain_huggingface import HuggingFaceEndpoint

# Import module for generating prompt templates.
from langchain.prompts import PromptTemplate

# Import module for generating few-shot prompt templates.
from langchain import FewShotPromptTemplate


# Bind Kaggle API
* make sure you've entered your Kaggle API keys into the COLAB secrets panel

In [18]:
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME') # Link to KAGGLE API secret key
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY') # Link to KAGGLE API secret key

# Bind HuggingFace API

In [19]:
hugging_face_api_token = userdata.get("huggingface_api_token_2")
login( token=hugging_face_api_token )

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Load the model through HuggingFace

In [31]:
# Try repo_id
#repo_id = "google/gemma-2b" # WORKS
#repo_id = "google/gemma-2-2b-it" # HF ACCESS FAILS
#repo_id = "google/gemma-2-2b" # HF ACCESS FAILS

# Try endpoint_url
endpoint_url = "https://api-inference.huggingface.co/models/google/gemma-2-2b"

llm = HuggingFaceEndpoint(huggingfacehub_api_token=hugging_face_api_token,
    # repo_id=repo_id,
    endpoint_url = endpoint_url,
    task="text-generation",
    max_new_tokens = 1024,
    do_sample=False
    # max_length=256, # seems to not be valid in the new HF interface
    #temperature = 0.7
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Define some few shot examples


In [32]:
examples =[
  {
    "prompt": "What is a variable in Python?",
    "target": "In Python, a variable is a named location used to store data values. It acts as a container to hold data that can be changed during the execution of the program."
  },
  {
    "prompt": "How do you define a function in Python?",
    "target": "To define a function in Python, you use the 'def' keyword followed by the function name and parentheses containing any parameters. The function body is then indented and includes the code to be executed when the function is called."
  },
  {
    "prompt": "What is a list comprehension in Python?",
    "target": "A list comprehension is a concise way to create lists in Python. It allows you to generate a new list by applying an expression to each item in an existing iterable, such as a list, tuple, or range."
  }
]

# Define the example format
* this is how each example is presented in the final prompt

In [33]:
example_template = """
User: {prompt}
AI: {target}
"""

# Define the example template
* this is langchain helper which will enable binding the named variables to the actual example data[link text](https://)

In [34]:
example_prompt = PromptTemplate(
    input_variables=['prompt', 'target'],
    template=example_template
)

# Define the prompt prefix and suffix

In [35]:
prefix = """The following are excerpts from conversations with an AI assistant focused on Python Programming.
The assistant is typically informative and encouraging, providing insightful and motivational responses to the user's questions about Python programming.
Here are some examples:
"""

suffix = """
User: {prompt}
AI: """

# Put it all together into the final prompt template

In [36]:
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["prompt"],
    example_separator="\n\n"
)

# Create an output parser

In [37]:
def output_parser(text):
    index = text.find("User:")
    if index != -1:
        return text[:index]
    else:
        return text

# Create a test prompt

In [38]:
prompt = "How do you split a string into a list using Python?"

full_prompt = few_shot_prompt_template.format(prompt=prompt)

print(full_prompt)

The following are excerpts from conversations with an AI assistant focused on Python Programming.
The assistant is typically informative and encouraging, providing insightful and motivational responses to the user's questions about Python programming.
Here are some examples:



User: What is a variable in Python?
AI: In Python, a variable is a named location used to store data values. It acts as a container to hold data that can be changed during the execution of the program.



User: How do you define a function in Python?
AI: To define a function in Python, you use the 'def' keyword followed by the function name and parentheses containing any parameters. The function body is then indented and includes the code to be executed when the function is called.



User: What is a list comprehension in Python?
AI: A list comprehension is a concise way to create lists in Python. It allows you to generate a new list by applying an expression to each item in an existing iterable, such as a list,

# Invoke the model on the prompt

In [39]:
response=llm.invoke(full_prompt)


HfHubHTTPError:  (Request ID: PcpuIJyZ1iOw3NZwENLzD)

403 Forbidden: None.
Cannot access content at: https://api-inference.huggingface.co/models/google/gemma-2-2b.
If you are trying to create or update content, make sure you have a token with the `write` role.
The model google/gemma-2-2b is too large to be loaded automatically (10GB > 10GB). Please use Spaces (https://huggingface.co/spaces) or Inference Endpoints (https://huggingface.co/inference-endpoints).

# Check the test response

In [ ]:
print(output_parser(response))